In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import pointbiserialr
from datetime import timedelta

# Function to dynamically identify health-related columns
def identify_health_labels(df, exclude_cols=['cow', 'date', 'IN_ALLEYS', 'REST', 'EAT', 'ACTIVITY_LEVEL', 'hour']):
    # Identify columns that are likely binary health indicators
    potential_health_cols = [col for col in df.columns if col not in exclude_cols]
    health_labels = [col for col in potential_health_cols 
                    if df[col].dropna().isin([0, 1]).all()]
    return health_labels

**Load the dataset**

In [ ]:
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    return df

### **Dataset First View**

In [ ]:
def display_dataset_head(df):
    return df.head()

### **Dataset Rows & Columns count**

In [ ]:
def print_dataset_shape(df):
    print("Number of rows are: ",df.shape[0])
    print("Number of columns are: ",df.shape[1])

**Displaying Column Names**

In [ ]:
def display_column_names(df):
    print(df.columns)

**Define behavioral and health-related features**

In [ ]:
def define_features(df):
    behavioral_features = ['IN_ALLEYS', 'REST', 'EAT', 'ACTIVITY_LEVEL']
    health_indicators = identify_health_labels(df)
    return behavioral_features, health_indicators

**Distribution of health states**

In [ ]:
def plot_health_states_distribution(df):
    health_states = identify_health_labels(df)
    if not health_states:
        print("No health states identified in the dataset.")
        return
    df_health = df[health_states]
    health_counts = df_health.sum().sort_values(ascending=False)
    total = len(df)
    health_percent = (health_counts / total) * 100

    safe_health_percent = health_percent.copy()
    perspective
safe_health_percent[safe_health_percent == 0] = 0.01

    plt.figure(figsize=(12, 6))
    bars = plt.bar(safe_health_percent.index, safe_health_percent.values, color='lightcoral')

    for i, bar in enumerate(bars):
        original = health_percent.values[i]
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
                 f'{original:.2f}%', ha='center', va='bottom')

    plt.yscale('log')
    plt.title('Proportion of Health States (Log Scale)', fontweight='bold')
    plt.xlabel('Health States', fontweight='bold')
    plt.ylabel('Percentage (log scale)', fontweight='bold')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7, which='both')
    plt.tight_layout()
    plt.show()

**Unique Value Counts per Event**

In [ ]:
def print_unique_value_counts(df):
    target_columns = identify_health_labels(df)
    print("Unique Value Counts per Event:")
    for col in target_columns:
        value_counts = df[col].value_counts()
        print(f"{col}:")
        print(value_counts)
        print("---------------------------------")

**Count the number of cows in the dataset**

In [ ]:
def count_unique_cows(df):
    num_cows = df['cow'].nunique()
    print(f"Total number of unique cows: {num_cows}")

**Display the ids of cows**

In [ ]:
def display_cow_ids(df):
    unique_cow_ids = df['cow'].unique()
    print("List of unique cow IDs:")
    print(unique_cow_ids)  # Prints all IDs as an array

    sorted_cow_ids = sorted(df['cow'].unique())
    print("\nSorted list of cow IDs:")
    print(sorted_cow_ids)

**Duplicate Values**

In [ ]:
def check_duplicates(df):
    # Dataset Duplicate Value Count
    dup = df.duplicated().sum()
    print(f'number of duplicated rows are {dup}')

**Missing Values**

In [ ]:
def check_missing_values(df):
    # Missing Values/Null Values Count
    return df.isnull().sum()

**Matrix of missing values**

In [ ]:
def plot_missing_values_matrix(file_path):
    data = pd.read_csv(file_path)

    plt.figure(figsize=(15, 8))
    msno.matrix(data, color=(0, 0.5, 0.5))  # Affichage des valeurs manquantes en matrice
    plt.title("Matrice des valeurs manquantes", fontsize=18, fontweight='bold')
    plt.show()

**Outliers detection**

Détails du calcul théorique :

1. Plage minimale ( -828 ) :

Si une vache passe 1 heure entière (3600 secondes) au repos : -0.23 * 3600 + 0.16 * 0 + 0.42 * 0 = -828

2. Plage maximale ( 1512 ) :

Si une vache passe 1 heure entière (3600 secondes) à manger : -0.23 * 0 + 0.16 * 0 + 0.42 * 3600 = 1512

In [ ]:
def detect_and_plot_outliers(df):
    min_theorique = -828
    max_theorique = 1512

    # Calculer les min/max observés
    min_obs = df['ACTIVITY_LEVEL'].min()
    max_obs = df['ACTIVITY_LEVEL'].max()

    print(f"Plage théorique : [{min_theorique}, {max_theorique}]")
    print(f"Plage observée  : [{min_obs:.2f}, {max_obs:.2f}]")

    # Identifier les valeurs hors plage
    outliers = df[(df['ACTIVITY_LEVEL'] < min_theorique) | (df['ACTIVITY_LEVEL'] > max_theorique)]
    num_outliers = len(outliers)

    plt.figure(figsize=(10, 6))
    plt.hist(df['ACTIVITY_LEVEL'], bins=50, color='skyblue', edgecolor='black')
    plt.axvline(x=min_theorique, color='red', linestyle='--', label='Borne min théorique')
    plt.axvline(x=max_theorique, color='green', linestyle='--', label='Borne max théorique')
    plt.xlabel('Niveau d\'activité')
    plt.ylabel('Fréquence')
    plt.title('Distribution des niveaux d\'activité')
    plt.legend()
    plt.xlim(-1000, 1600)  # Pour mieux visualiser les éventuels outliers
    plt.show()

    print("\n[ Rapport d'anomalies ]")
    print(f"- Valeurs hors plage théorique : {num_outliers} ({num_outliers/len(df)*100:.2f}% du dataset)")

    if num_outliers > 0:
        print("- Exemples de lignes problématiques :")
        print(outliers[['date', 'ACTIVITY_LEVEL']].head())
    else:
        print("- Aucune valeur hors plage détectée.")

In [ ]:
def plot_histograms(df):
    # Plot histograms for the selected columns to understand their distributions
    cols = ['IN_ALLEYS', 'REST', 'EAT', 'ACTIVITY_LEVEL']
    df[cols].hist(bins=20, figsize=(12, 10))
    plt.suptitle("Data Distribution")
    plt.show()

**Visualization of Health States Proportions**

In [ ]:
def plot_health_states_proportions(file_path):
    df = pd.read_csv(file_path)
    health_states = identify_health_labels(df)
    if not health_states:
        print("No health states identified in the dataset.")
        return

    df_health = df[health_states]
    health_counts = df_health.sum().sort_values(ascending=False)
    total = len(df)
    health_percent = (health_counts / total) * 100

    plt.figure(figsize=(12, 6))
    bars = plt.bar(health_percent.index, health_percent.values, color='lightcoral')

    # Formatting: 2 decimal places
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.2f}%',  # .2f = 2 decimal places
                 ha='center', va='bottom')

    plt.title('Proportion of Health States (in %)', fontweight='bold')
    plt.xlabel('Health States', fontweight='bold')
    plt.ylabel('Percentage', fontweight='bold')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

**Missing hours**

In [ ]:
def plot_missing_hours(file_path):
    # Load the dataset
    df = pd.read_csv(file_path)

    # Ensure the datetime column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Count the number of observations (rows) per cow per day
    obs_counts = df.groupby(['cow', 'date']).size().reset_index(name='hour_count')

    # Identify rows with missing hours (less than 24)
    missing_obs = obs_counts[obs_counts['hour_count'] < 24]

    # Create a plot showing missing hour count per cow
    plt.figure(figsize=(12, 6))
    missing_obs.groupby('cow')['hour_count'].apply(lambda x: (24 - x).sum()).sort_values(ascending=False).plot(kind='bar')
    plt.title("Total Missing Hours per Cow (only cows with missing 24h records)")
    plt.xlabel("Cow ID")
    plt.ylabel("Total Missing Hours")
    plt.tight_layout()
    plt.show()

**Check Date Range and Missing Dates**

In [ ]:
def check_date_range(df):
    # Convert 'date' to datetime
    df['date'] = pd.to_datetime(df['date'])

    # Get min/max dates
    min_date = df['date'].min()
    max_date = df['date'].max()
    print(f"Date range: {min_date} to {max_date}")

    # Generate all expected dates in the range
    all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
    print(f"Total expected days: {len(all_dates)}")

    # Check missing dates
    missing_dates = all_dates.difference(df['date'].unique())
    print(f"\nMissing dates {len(missing_dates)}:", ', '.join(missing_dates.strftime('%Y-%m-%d')))

**Analysis of Cows with Multiple Diseases**

In [ ]:
def analyze_multiple_diseases(df):
    maladie_cols = [col for col in identify_health_labels(df) if col != 'OK']
    if not maladie_cols:
        print("No disease columns identified in the dataset.")
        return

    df_non_ok = df[df['OK'] == 0].copy()

    df_non_ok['nb_maladies'] = df_non_ok[maladie_cols].sum(axis=1)

    # Keep only rows with more than one disease
    multi_maladie = df_non_ok[df_non_ok['nb_maladies'] > 1].copy()

    # Extract the names of the diseases present
    def get_maladies_presentes(row):
        return [col for col in maladie_cols if row[col] == 1]

    multi_maladie['maladies_presentes'] = multi_maladie.apply(get_maladies_presentes, axis=1)

    # Display results
    print(f"Number of cases with multiple diseases: {len(multi_maladie)}")
    print(multi_maladie[['cow', 'date', 'hour', 'maladies_presentes']])

**Investigate the correlation between the "OK" label and other health states**

In [ ]:
def check_ok_label_consistency(df):
    health_labels = [col for col in identify_health_labels(df) if col != 'OK']
    if not health_labels:
        print("No health labels (excluding OK) identified in the dataset.")
        return

    violations = df[(df['OK'] == 1) & (df[health_labels].sum(axis=1) > 0)]

    if violations.empty:
        print("✅ Aucun cas trouvé où OK == 1 et un état de santé est aussi à 1. Tout est cohérent.")
    else:
        print("⚠️ Des cas incohérents ont été trouvés ! Voici les lignes concernées :")
        print(violations)

**Correlation Analysis**

In [ ]:
def perform_correlation_analysis(df):
    df = df.drop(columns=[col for col in ['cow', 'date', 'LPS', 'acidosis', 'ACTIVITY_LEVEL', 'hour'] if col in df.columns])

    # Define columns
    behavioral_features = ['IN_ALLEYS', 'REST', 'EAT']
    health_features = identify_health_labels(df)
    if not health_features:
        print("No health features identified in the dataset.")
        return

    # --- 1. Spearman Correlation between Behavioral Features ---
    behavior_corr = df[behavioral_features].corr(method='spearman')

    plt.figure(figsize=(6, 4))
    sns.heatmap(behavior_corr, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Spearman Correlation - Behavioral Features")
    plt.show()

    # --- 2. Pearson Correlation (Phi) between Health Features ---
    health_corr = df[health_features].corr(method='pearson')

    plt.figure(figsize=(10, 8))
    sns.heatmap(health_corr, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Phi Correlation (Pearson) - Health Features")
    plt.show()

    # --- 3. Point-Biserial Correlation: Behavioral vs Health States ---
    point_biserial_matrix = pd.DataFrame(index=health_features, columns=behavioral_features)

    for h in health_features:
        for b in behavioral_features:
            valid_data = df[[b, h]].dropna()
            if valid_data[h].nunique() > 1:  # Ensure variable is not constant
                corr, _ = pointbiserialr(valid_data[h], valid_data[b])
                point_biserial_matrix.loc[h, b] = corr
            else:
                point_biserial_matrix.loc[h, b] = np.nan

    point_biserial_matrix = point_biserial_matrix.astype(float)

    plt.figure(figsize=(8, 6))
    sns.heatmap(point_biserial_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Point-Biserial Correlation - Behavioral vs Health Features")
    plt.xlabel("Behavioral Features")
    plt.ylabel("Health Features")
    plt.show()

In [ ]:
def plot_point_biserial_correlation(df):
    # Define columns
    behavioral_features = ['IN_ALLEYS', 'REST', 'EAT']
    health_features = identify_health_labels(df)
    if not health_features:
        print("No health features identified in the dataset.")
        return

    # Collect correlations
    correlation_data = []

    for h in health_features:
        for b in behavioral_features:
            valid_data = df[[b, h]].dropna()
            if valid_data[h].nunique() > 1:
                corr, pval = pointbiserialr(valid_data[h], valid_data[b])
                correlation_data.append({
                    'Health State': h,
                    'Behavioral Feature': b,
                    'Correlation': corr,
                    'p-value': pval
                })

    # Convert to DataFrame
    corr_df = pd.DataFrame(correlation_data)

    # Plot
    plt.figure(figsize=(8, 5))
    for health_state in corr_df["Health State"].unique():
        subset = corr_df[corr_df["Health State"] == health_state]
        plt.bar(subset["Behavioral Feature"] + f" ({health_state})", subset["Correlation"], label=health_state)

    plt.axhline(0, color='gray', linestyle='--')
    plt.title("Point-Biserial Correlation: Behavioral vs Health States")
    plt.ylabel("Correlation Coefficient")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
def plot_phi_correlation(df):
    if 'oestrus' in df.columns and 'OK' in df.columns:
        phi_corr = df[["oestrus", "OK"]].corr(method="pearson").iloc[0, 1]

        # Prepare plot
        plt.figure(figsize=(6, 3))
        x = ["oestrus vs OK"]
        y = [phi_corr]

        # Lollipop style plot
        plt.plot(x, y, marker='o', linestyle='-', color='steelblue')
        plt.axhline(0, color='gray', linestyle='--')
        plt.ylim(-1, 1)
        plt.title("Phi Correlation Between Binary Health Features")
        plt.ylabel("Correlation Coefficient")
        plt.grid(axis='y', linestyle=':')
        plt.tight_layout()
        plt.show()
    else:
        print("Columns 'oestrus' or 'OK' not found in the dataset.")

In [ ]:

datasets = ['dataset1.csv', 'dataset2.csv', 'dataset3.csv', 'dataset4.csv']  
for file_path in datasets:
    print(f"\nAnalyzing {file_path}")
    df = load_dataset(file_path)
    print("\nDataset First View:")
    display_dataset_head(df)
    print("\nDataset Rows & Columns count:")
    print_dataset_shape(df)
    print("\nColumn Names:")
    display_column_names(df)
    behavioral_features, health_indicators = define_features(df)
    print("\nDistribution of Health States:")
    plot_health_states_distribution(df)
    print("\nUnique Value Counts per Event:")
    print_unique_value_counts(df)
    print("\nNumber of Unique Cows:")
    count_unique_cows(df)
    print("\nCow IDs:")
    display_cow_ids(df)
    print("\nDuplicate Values:")
    check_duplicates(df)
    print("\nMissing Values:")
    print(check_missing_values(df))
    print("\nMatrix of Missing Values:")
    plot_missing_values_matrix(file_path)
    print("\nOutliers Detection:")
    detect_and_plot_outliers(df)
    print("\nData Distribution Histograms:")
    plot_histograms(df)
    print("\nHealth States Proportions:")
    plot_health_states_proportions(file_path)
    print("\nMissing Hours:")
    plot_missing_hours(file_path)
    print("\nDate Range and Missing Dates:")
    check_date_range(df)
    print("\nAnalysis of Cows with Multiple Diseases:")
    analyze_multiple_diseases(df)
    print("\nOK Label Consistency Check:")
    check_ok_label_consistency(df)
    print("\nCorrelation Analysis:")
    perform_correlation_analysis(df)
    print("\nPoint-Biserial Correlation Plot:")
    plot_point_biserial_correlation(df)
    print("\nPhi Correlation:")
    plot_phi_correlation(df)